## Day 21: Fractal Art

http://adventofcode.com/2017/day/21

### Part 1

First create a big dictionary of transformations, including rotations and flips and flips of rotations. That should cover everything. `numpy` has a lot of this built in.

In [1]:
import numpy as np
import math

def read_pattern(p):
    return '\n'.join(p.split('/'))

test_pattern = read_pattern('##../#.../..../..#.')

print(test_pattern)

##..
#...
....
..#.


In [2]:
def pattern_to_matrix(p):
    return np.array([[1 if c == '#' else 0 for c in s] 
                      for s in p.splitlines()])

pattern_to_matrix(test_pattern)

array([[1, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0]])

In [3]:
def matrix_to_pattern(m):
    return '\n'.join([''.join(['#' if b == 1 else '.' for b in r]) 
                      for r in m.tolist()])

print(matrix_to_pattern(pattern_to_matrix(test_pattern)))

##..
#...
....
..#.


In [4]:
from functools import partial

rotations = [lambda x: x, np.rot90, partial(np.rot90, k=2), partial(np.rot90, k=3)]
flips = [lambda x: x, np.fliplr, np.flipud]

In [5]:
test_3x3 = np.arange(1, 10)
test_3x3.shape = (3, 3)

for r in rotations:
    for f in flips:
        print(f(r(test_3x3)), '\n')

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[3 2 1]
 [6 5 4]
 [9 8 7]] 

[[7 8 9]
 [4 5 6]
 [1 2 3]] 

[[3 6 9]
 [2 5 8]
 [1 4 7]] 

[[9 6 3]
 [8 5 2]
 [7 4 1]] 

[[1 4 7]
 [2 5 8]
 [3 6 9]] 

[[9 8 7]
 [6 5 4]
 [3 2 1]] 

[[7 8 9]
 [4 5 6]
 [1 2 3]] 

[[3 2 1]
 [6 5 4]
 [9 8 7]] 

[[7 4 1]
 [8 5 2]
 [9 6 3]] 

[[1 4 7]
 [2 5 8]
 [3 6 9]] 

[[9 6 3]
 [8 5 2]
 [7 4 1]] 



That's covered everything and then some, e.g. rotate 180 then flip left to right == rotate 0 then flip up to down. I'll just shovel everything into a set.

In [6]:
def transformations(pattern):
    m = pattern_to_matrix(pattern)
    return {matrix_to_pattern(f(r(m))) 
            for r in rotations for f in flips}

In [7]:
for p in transformations(test_pattern):
    print(p, '\n')

....
...#
#...
##.. 

..#.
....
#...
##.. 

##..
#...
....
..#. 

.#..
....
...#
..## 

....
#...
...#
..## 

..##
...#
#...
.... 

..##
...#
....
.#.. 

##..
#...
...#
.... 



In [8]:
def process_data_line(line):
    k, v = line.strip().split(' => ')
    key = read_pattern(k)
    value = read_pattern(v)
    
    return {transformation: value for transformation in transformations(key)}

test_data = '''../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
'''.splitlines()

process_data_line(test_data[0])

{'#.\n..': '##.\n#..\n...',
 '.#\n..': '##.\n#..\n...',
 '..\n#.': '##.\n#..\n...',
 '..\n.#': '##.\n#..\n...'}

In [9]:
def build_dictionary(data):
    dictionary = {}
    
    for line in data:
        dictionary.update(process_data_line(line))
        
    return dictionary

test_dictionary = build_dictionary(test_data)

test_dictionary

{'###\n#..\n.#.': '#..#\n....\n....\n#..#',
 '###\n..#\n.#.': '#..#\n....\n....\n#..#',
 '##.\n#.#\n#..': '#..#\n....\n....\n#..#',
 '#.\n..': '##.\n#..\n...',
 '#..\n#.#\n##.': '#..#\n....\n....\n#..#',
 '.#\n..': '##.\n#..\n...',
 '.##\n#.#\n..#': '#..#\n....\n....\n#..#',
 '.#.\n#..\n###': '#..#\n....\n....\n#..#',
 '.#.\n..#\n###': '#..#\n....\n....\n#..#',
 '..\n#.': '##.\n#..\n...',
 '..\n.#': '##.\n#..\n...',
 '..#\n#.#\n.##': '#..#\n....\n....\n#..#'}

In [10]:
def enhance(book, pattern):
    # Make one enhancement using the given rulebook
    
    length = len(pattern.splitlines()[0])
    split_length = 2 if length % 2 == 0 else 3
    # nxn subsquares
    n = length // split_length
    
    
    # Turn pattern into matrix and split into smaller squares 
    m = pattern_to_matrix(pattern)
    split_matrix = [np.split(s, n, axis=1) for s in np.split(m, n)]
    
    
    # The evolved squares            
    enhancements = [[book[matrix_to_pattern(split_matrix[i][j])] 
                     for j in range(n)] 
                    for i in range(n)]
    
    
    # Stitch them back together
    rows = [np.concatenate([pattern_to_matrix(p) for p in enhancements[i]], axis=1) 
            for i in range(n)]
    return matrix_to_pattern(np.concatenate(rows))

No, that didn't work first time. On reflection I shouldn't have kept switching between numpy arrays and strings but I thought it would be useful for debugging and hashing.

In [11]:
starting_pattern = '''.#.
..#
###'''
    
p = starting_pattern
for i in range(1, 3):
    print(i)
    p = enhance(test_dictionary, p)
    print(p)

1
#..#
....
....
#..#
2
##.##.
#..#..
......
##.##.
#..#..
......


That seems to be working.

In [12]:
with open('input', 'r') as f:
    problem_book = build_dictionary(f.readlines())
    
p = starting_pattern
for i in range(1, 6):
    print(i)
    p = enhance(problem_book, p)
    print(p)

1
#.##
#.#.
..#.
..##
2
##.#.#
.#....
##...#
#..#.#
##....
.....#
3
#.#..#..#
....#..#.
..#..#..#
#.#..###.
....#..#.
..#..###.
##.#....#
.#.##..#.
##......#
4
.#...###.###
##...###.###
.#..#.###.##
.###.#...#..
.#...###.#.#
##...###.###
.#..#.#####.
.###.#...#.#
.#.#.##..###
.###.###.###
###.....#.##
.#.#.#.#.#..
5
#.##..##..#.##..#.
...##..#.###.#.###
..#...##....##....
##.##...###...###.
.#..#..##.#..##.#.
##.##....##....##.
#.##..##..#.##.#.#
...##..#.###.#....
..#...##....##...#
##.##...###.#.#..#
.#..#..##.#.....##
##.##....##...#...
##.#.###.#.###..#.
.#.....#.....#.###
##...###...###....
#.#..#..#..#..###.
....##.#..#..##.#.
..#.....#..#...##.


In [13]:
p.count('#')

142

Phew.

### Part 2

In [14]:
p = starting_pattern
for i in range(1, 19):
    print(i)
    p = enhance(problem_book, p)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [15]:
p.count('#')

1879071